In [4]:
import requests
from bs4 import BeautifulSoup

# Check the actual structure of the fantasy site
url = "https://www.futmondo.com/"  # or specific endpoint
response = requests.get(url)

# Or check if they have an undocumented API
# Use browser DevTools → Network → XHR to find JSON endpoints

In [5]:
response

<Response [200]>

In [6]:
import requests
from bs4 import BeautifulSoup

url = "https://www.futmondo.com/"
response = requests.get(url)

print(f"Status Code: {response.status_code}")
print(f"Content Length: {len(response.content)}")

# Parse the HTML
soup = BeautifulSoup(response.content, 'html.parser')

# See the structure
print(soup.prettify()[:2000])  # First 2000 characters

# Find all links to understand site structure
links = soup.find_all('a', href=True)
for link in links[:20]:  # First 20 links
    print(link['href'])

Status Code: 200
Content Length: 15700
<!DOCTYPE HTML>
<html lang="es">
 <head>
  <title>
   futmondo - Fútbol fantasy manager  - futmondo
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="VB8XORVWTq7eEzHSyxjrYJ1wHxA" name="alexaVerifyID">
   <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
   <link href="//www.futmondo.com/humans.txt" rel="author" type="text/plain"/>
   <link href="/img/favicon.ico" rel="shortcut icon"/>
   <link href="/img/icon-iphone.png" rel="apple-touch-icon"/>
   <link href="/img/icon-ipad.png" rel="apple-touch-icon" sizes="72x72"/>
   <link href="/img/icon-iphone4.png" rel="apple-touch-icon" sizes="114x114"/>
   <meta content="futmondo - Fantasy Manager de futbol basado en la actuaciones reales de los jugadores" name="description"/>
   <meta content="futmondo, fantasy manager, liga santander, premier league" name="keywords"/>
   <meta content="31652920841

In [7]:
import requests
import json

# Common patterns for fantasy sites
base_url = "https://www.futmondo.com"

# Try common API endpoints (these are examples, need to verify)
possible_endpoints = [
    "/api/players",
    "/api/v1/players",
    "/api/market",
    "/api/league/laliga",
    "/data/players.json",
]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

for endpoint in possible_endpoints:
    try:
        r = requests.get(base_url + endpoint, headers=headers)
        if r.status_code == 200:
            print(f"✓ Found: {endpoint}")
            print(f"Response preview: {r.text[:200]}")
            print("-" * 50)
    except:
        pass

✓ Found: /api/market
Response preview: <!DOCTYPE HTML>
<html lang="es">
<head>
    <title> futmondo - Fútbol fantasy manager  - futmondo</title>
    
    
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
        
--------------------------------------------------


In [8]:
import requests
import json

# The actual app domain
base_url = "https://app.futmondo.com"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'application/json',
}

# Test if we can access the app
response = requests.get(base_url, headers=headers)
print(f"App Status: {response.status_code}")
print(f"Content preview: {response.text[:500]}")

App Status: 200
Content preview: <!DOCTYPE html>
<html>
  <head>
    <base href="/">

    <meta charset="UTF-8">
    <meta content="IE=Edge" http-equiv="X-UA-Compatible">
    <meta name="description" content="¡Únete gratis y dirige tu equipo en el mánager de fútbol más completo del mercado!">

    <!-- iOS meta tags & icons -->
    <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">
    <meta name="apple-mobile-web-app-title" content="futmondo">
    <l


In [10]:
import requests
import re
import json

url = "https://app.futmondo.com"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

response = requests.get(url, headers=headers)

# Find JavaScript file URLs
js_urls = re.findall(r'src="(.*?\.js)"', response.text)
print("JavaScript files found:")
for js_url in js_urls:
    if not js_url.startswith('http'):
        # Fix: properly join URLs
        if js_url.startswith('/'):
            full_url = 'https://app.futmondo.com' + js_url
        else:
            full_url = 'https://app.futmondo.com/' + js_url
    else:
        full_url = js_url
    print(f"  {full_url}")

# Download and search the MAIN app.js file for API endpoints
print("\nSearching for Futmondo's main app.js file...")

# Look specifically for the app.js file
for js_url in js_urls:
    if 'app.js' in js_url.lower() and 'facebook' not in js_url and 'firebase' not in js_url:
        if not js_url.startswith('http'):
            if js_url.startswith('/'):
                main_js_url = 'https://app.futmondo.com' + js_url
            else:
                main_js_url = 'https://app.futmondo.com/' + js_url
        else:
            main_js_url = js_url
            
        print(f"\nDownloading: {main_js_url}")
        
        try:
            js_response = requests.get(main_js_url, headers=headers, timeout=10)
            js_content = js_response.text
            
            print(f"File size: {len(js_content)} characters")
            
            # Search for API endpoint patterns
            print("\n" + "="*60)
            print("Searching for API endpoints...")
            
            # Look for various API patterns
            patterns_to_search = [
                (r'["\']https?://[^"\']*api[^"\']*["\']', "API URLs"),
                (r'["\']https?://[^"\']*data[^"\']*["\']', "Data URLs"),
                (r'baseURL["\']?\s*:\s*["\']([^"\']+)["\']', "Base URL"),
                (r'apiUrl["\']?\s*:\s*["\']([^"\']+)["\']', "API URL"),
                (r'/api/v?\d*/\w+', "API endpoints"),
            ]
            
            for pattern, description in patterns_to_search:
                matches = re.findall(pattern, js_content, re.IGNORECASE)
                if matches:
                    print(f"\n{description}:")
                    for match in set(list(matches)[:10]):  # Show first 10 unique
                        print(f"  {match}")
            
            # Also look for hardcoded domain names
            print("\n" + "="*60)
            print("Looking for domain names...")
            domains = re.findall(r'https?://([a-zA-Z0-9.-]+\.futmondo\.com)', js_content)
            domains += re.findall(r'https?://([a-zA-Z0-9.-]+\.mondocore\.com)', js_content)
            domains += re.findall(r'https?://([a-zA-Z0-9.-]+\.mondosports\.co\.uk)', js_content)
            
            if domains:
                print("Domains found:")
                for domain in set(domains):
                    print(f"  {domain}")
            
            break  # Stop after processing the main app.js
            
        except Exception as e:
            print(f"Error downloading {main_js_url}: {e}")

JavaScript files found:
  https://app.futmondo.com/app.js
  https://connect.facebook.net/en_US/sdk.js
  https://connect.facebook.net/en_US/sdk.js
  https://www.gstatic.com/firebasejs/8.10.0/firebase-app.js
  https://www.gstatic.com/firebasejs/8.10.0/firebase-auth.js
  https://www.gstatic.com/firebasejs/8.10.0/firebase-firestore.js
  https://www.gstatic.com/firebasejs/8.10.0/firebase-messaging.js
  https://www.gstatic.com/firebasejs/8.10.0/firebase-analytics.js
  https://www.google.com/recaptcha/api.js

Searching for Futmondo's main app.js file...

Downloading: https://app.futmondo.com/app.js
File size: 854 characters

Searching for API endpoints...

Looking for domain names...


In [14]:
import requests
import json
import time

# --- 1. CONFIGURATION (Verified Data) ---

BASE_URL = 'https://api.futmondo.com'

# CONFIRMED VALUES from your account data
YOUR_AUTH_TOKEN = '5e65_20f468a19ad19ef73979642df99d6603' 
YOUR_USER_ID = '56c6b62085617f9b1dc7d061'
YOUR_CHAMPIONSHIP_ID = '5f7b19924dcd0d43e8a092dd4'
YOUR_USERTEAM_ID = '5f7b33ea6f9120324dedcb15'

# --- 2. ENDPOINTS & HEADERS ---

ENDPOINTS = {
    'GET_TEAM_PLAYERS': '/1/userteam/roster',
}

# The final, correct set of headers to satisfy API security checks
HEADERS = {
    'Authorization': f'Bearer {YOUR_AUTH_TOKEN}', 
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'x-futmondo-token': YOUR_AUTH_TOKEN,       # Custom required header
    'x-futmondo-userid': YOUR_USER_ID,         # Custom required header
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

# --- 3. CORE REQUEST FUNCTION ---

In [15]:
def make_request(endpoint_name, payload):
    """Makes API request and handles the response."""
    
    endpoint = ENDPOINTS.get(endpoint_name)
    if not endpoint:
        print(f"❌ ERROR: Unknown endpoint name {endpoint_name}")
        return None

    full_url = f"{BASE_URL}{endpoint}"
    
    print(f"\n{'='*70}")
    print(f"📡 {endpoint_name}")
    print(f"{'='*70}")
    print(f"URL: {full_url}")
    print(f"Payload: {json.dumps(payload, indent=2)}")

    response = requests.post(
        full_url,
        headers=HEADERS, # Use the global HEADERS dictionary
        json=payload,
        timeout=20
    )
    
    print(f"Status: {response.status_code}")
    response.raise_for_status() 

    data = response.json()
    
    # Safely handle the response structure (list vs. dict, and checking for API errors)
    if isinstance(data, dict):
        answer = data.get('answer', data)
        if isinstance(answer, dict) and answer.get('error'):
            print(f"❌ API ERROR: {answer.get('code', 'Unknown Error')}")
            return None
        
        # Successful dict or list within the 'answer' key
        if isinstance(answer, list):
            print(f"✅ SUCCESS - Got {len(answer)} items in answer")
        
        return data
        
    elif isinstance(data, list):
        print(f"✅ SUCCESS - Got {len(data)} items directly")
        return data

    return data

In [16]:
roster_payload = {
    'query': {
        'championshipId': YOUR_CHAMPIONSHIP_ID,
        'userteamId': YOUR_USERTEAM_ID
    }
}

# Execute the Roster request
roster_data_response = make_request('GET_TEAM_PLAYERS', roster_payload)


📡 GET_TEAM_PLAYERS
URL: https://api.futmondo.com/1/userteam/roster
Payload: {
  "query": {
    "championshipId": "5f7b19924dcd0d43e8a092dd4",
    "userteamId": "5f7b33ea6f9120324dedcb15"
  }
}
Status: 200
✅ SUCCESS - Got 0 items in answer


In [17]:
roster_data_response

{'answer': [],
 'query': {'championshipId': '5f7b19924dcd0d43e8a092dd4',
  'userteamId': '5f7b33ea6f9120324dedcb15'},
 'header': {}}

In [ ]:
if roster_data_response:
    print("\n👥 ROSTER DATA SUCCESSFULLY RECEIVED:")
    
    # Extract the list of players, which is often under the 'answer' key
    player_list = roster_data_response.get('answer', roster_data_response)

    if isinstance(player_list, list) and len(player_list) > 0:
        print(f"Found {len(player_list)} players.")
        print("\n--- SAMPLE PLAYER DATA ---")
        print(json.dumps(player_list[0], indent=2))
    elif isinstance(player_list, list) and len(player_list) == 0:
        print("⚠️ WARNING: Roster request successful, but the player list is empty.")
    else:
        print(json.dumps(roster_data_response, indent=2))
else:
    print("\n❌ FAILED to retrieve roster data. Check the error message above.")

print("\n" + "="*70)
print("📋 TASK COMPLETE")
print("="*70)

In [1]:
import requests

# Your credentials
AUTH_TOKEN = '5e65_20f468a19ad19ef73979642df99d6603'
USER_ID = '56c6b62085617f9b1dc7d061'
CHAMPIONSHIP_ID = '5f7b19924dcd043e8a092dd4'  # FIXED THE TYPO

headers = {
    'Authorization': f'Bearer {AUTH_TOKEN}',
    'x-futmondo-token': AUTH_TOKEN,
    'x-futmondo-userid': USER_ID,
}

# Test: Get all players
response = requests.post(
    'https://api.futmondo.com/1/market/players',
    headers=headers,
    json={'query': {'championshipId': CHAMPIONSHIP_ID}}
)

data = response.json()

if 'answer' in data:
    players = data['answer']
    print(f"✅ SUCCESS: Got {len(players)} players")
    print(f"\nFirst player: {players[0]}")
else:
    print("❌ FAILED")
    print(data)

✅ SUCCESS: Got 2 players


KeyError: 0

In [4]:
import requests

# Your credentials
AUTH_TOKEN = '5e65_20f468a19ad19ef73979642df99d6603'
USER_ID = '56c6b62085617f9b1dc7d061'
CHAMPIONSHIP_ID = '5f7b19924dcd043e8a092dd4'
USERTEAM_ID = '5f7b33ea6f9120324dedcb15'

headers = {
    'Authorization': f'Bearer {AUTH_TOKEN}',
    'x-futmondo-token': AUTH_TOKEN,
    'x-futmondo-userid': USER_ID,
}

# Test: Get all players WITH userteamId
response = requests.post(
    'https://api.futmondo.com/1/market/players',
    headers=headers,
    json={'query': {
        'championshipId': CHAMPIONSHIP_ID,
        'userteamId': USERTEAM_ID
    }}
)

data = response.json()

if 'answer' in data and isinstance(data['answer'], list):
    players = data['answer']
    print(f"✅ Got {len(players)} players")
    if len(players) > 0:
        print(f"\nFirst player: {players[0].get('name')} - {players[0].get('team')}")
else:
    print(data)

✅ Got 21 players

First player: Koke - Atlético de Madrid


In [10]:
import requests

AUTH_TOKEN = '5e65_20f468a19ad19ef73979642df99d6603'
USER_ID = '56c6b62085617f9b1dc7d061'
CHAMPIONSHIP_ID = '5f7b19924dcd043e8a092dd4'

headers = {
    'Authorization': f'Bearer {AUTH_TOKEN}',
    'x-futmondo-token': AUTH_TOKEN,
    'x-futmondo-userid': USER_ID,
}

# Try different endpoints
endpoints_to_try = [
    '/1/championship/players',
    '/3/championship/players',
    '/2/championship/roster',
    '/1/championship/roster',
]

for endpoint in endpoints_to_try:
    print(f"\nTrying: {endpoint}")
    response = requests.post(
        f'https://api.futmondo.com{endpoint}',
        headers=headers,
        json={'query': {'championshipId': CHAMPIONSHIP_ID}}
    )
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        data = response.json()
        if 'answer' in data:
            print(f"✅ WORKS! Got: {list(data['answer'].keys())}")
            break


Trying: /1/championship/players
Status: 404

Trying: /3/championship/players
Status: 404

Trying: /2/championship/roster
Status: 404

Trying: /1/championship/roster
Status: 404


In [11]:
import requests

AUTH_TOKEN = '5e65_20f468a19ad19ef73979642df99d6603'
USER_ID = '56c6b62085617f9b1dc7d061'
CHAMPIONSHIP_ID = '5f7b19924dcd043e8a092dd4'

headers = {
    'Authorization': f'Bearer {AUTH_TOKEN}',
    'x-futmondo-token': AUTH_TOKEN,
    'x-futmondo-userid': USER_ID,
}

# Get data for ONE specific player
player_id = '504e58bb4d8bec9a67000187'  # Koke's ID from your earlier data

response = requests.post(
    'https://api.futmondo.com/1/player/summary',
    headers=headers,
    json={'query': {
        'playerId': player_id,
        'championshipId': CHAMPIONSHIP_ID
    }}
)

data = response.json()
print(data)

{'answer': {'data': {'id': '504e58bb4d8bec9a67000187', 'name': 'Koke', 'slug': '67047092', 'role': 'centrocampista', 'role2': '', 'photo': '67047092.png', 'points': 61, 'value': 21049633, 'team': 'Atlético de Madrid', 'logo': 'atletico-de-madrid.png', 'teamId': '504e581e4d8bec9a670000c8', 'status': '', 'rating': 5, 'average': {'average': 5.083333333333333, 'homeAverage': 5.833333333333333, 'awayAverage': 4.333333333333333, 'averageLastFive': 4.4, 'matches': 12, 'fitness': [2, 8, 5, 5, 2]}, 'x': 'adidas2', 'change': 85457, 'computer': False, 'releaseDate': '2025-11-08T05:21:59.056Z', 'total': {'points': 61, 'played': 12}}, 'prices': [{'_id': '691d38aeee5bde071c21543a', 'c': 8778286, 's': 28699277, 'date': '2025-11-19T03:25:34.352Z', 'price': 20280310}, {'_id': '691e8a41db7f0a06fc2200ec', 'c': 8775886, 's': 29409658, 'date': '2025-11-20T03:25:53.202Z', 'price': 20531305}, {'_id': '691fdbc5b349de06f1dda750', 'c': 8773505, 's': 30345652, 'date': '2025-11-21T03:25:57.419Z', 'price': 2085802

In [13]:
import requests
import json
import time

AUTH_TOKEN = '5e65_20f468a19ad19ef73979642df99d6603'
USER_ID = '56c6b62085617f9b1dc7d061'
CHAMPIONSHIP_ID = '5f7b19924dcd043e8a092dd4'
USERTEAM_ID = '5f7b33ea6f9120324dedcb15'

headers = {
    'Authorization': f'Bearer {AUTH_TOKEN}',
    'x-futmondo-token': AUTH_TOKEN,
    'x-futmondo-userid': USER_ID,
}

# STEP 1: Get all player IDs from market
response = requests.post(
    'https://api.futmondo.com/1/market/players',
    headers=headers,
    json={'query': {
        'championshipId': CHAMPIONSHIP_ID,
        'userteamId': USERTEAM_ID
    }}
)

market_data = response.json()
all_players = market_data['answer']

print(f"Found {len(all_players)} players")

# STEP 2: Get detailed data for each player
detailed_players = []

for i, player in enumerate(all_players):
    player_id = player['id']
    player_name = player['name']
    
    print(f"{i+1}/{len(all_players)}: {player_name}...")
    
    response = requests.post(
        'https://api.futmondo.com/1/player/summary',
        headers=headers,
        json={'query': {
            'playerId': player_id,
            'championshipId': CHAMPIONSHIP_ID
        }}
    )
    
    if response.status_code == 200:
        detailed_players.append(response.json())
    
    time.sleep(0.5)  # Don't spam the API

# Save everything
with open('all_detailed_players.json', 'w', encoding='utf-8') as f:
    json.dump(detailed_players, f, indent=2, ensure_ascii=False)

print(f"\n✅ Done! Saved {len(detailed_players)} players")

Found 21 players
1/21: Koke...
2/21: Álvaro Lemos...
3/21: Rubén García...
4/21: Oriol Rey...
5/21: Güler...
6/21: Rodri Mendoza...
7/21: Cardoso...
8/21: Domingos Duarte...
9/21: Salinas...
10/21: Courtois...
11/21: Morlanes...
12/21: Espí...
13/21: Jutglà...
14/21: Primo...
15/21: Eyong...
16/21: Gueye...
17/21: Jon Pacheco...
18/21: Chust...
19/21: Román...
20/21: Antony...
21/21: N. González...

✅ Done! Saved 21 players
